In [61]:
import cv2
import mediapipe as mp
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import mediapipe as mp
import pickle


In [ ]:
import cv2
import numpy as np
import os
import mediapipe as mp

# MediaPipe initialization for hand landmarks
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=2,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)

def extract_keypoints_from_frame(frame):
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    
    keypoints = []
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for landmark in hand_landmarks.landmark:
                keypoints.append([landmark.x, landmark.y])
                
    expected_num_keypoints = 21  # MediaPipe Hands provides 21 landmarks per hand
    if len(keypoints) < expected_num_keypoints:
        keypoints += [[0, 0]] * (expected_num_keypoints - len(keypoints))
        
    return np.array(keypoints)[:expected_num_keypoints], len(keypoints)  # Return keypoints and count

def extract_keypoints_from_video(video_path):
    keypoints = []
    total_frames = 0
    frames_with_keypoints = 0
    cap = cv2.VideoCapture(video_path)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        total_frames += 1
        frame_keypoints, count = extract_keypoints_from_frame(frame)
        keypoints.append(frame_keypoints)
        
        if count > 0:
            frames_with_keypoints += 1
            
    cap.release()
    cv2.destroyAllWindows()
    
    print(f"Video: {video_path} - Total frames: {total_frames}, Frames with keypoints: {frames_with_keypoints}")
    
    return np.array(keypoints)

def load_videos_from_directory(directory):
    data = {}
    print(f"Loading videos from directory: {directory}")
    
    for video_file in os.listdir(directory):
        if video_file.endswith('.mp4'):
            video_path = os.path.join(directory, video_file)
            print(f"Loading video: {video_path}")
            keypoints = extract_keypoints_from_video(video_path)
            
            # Extract the label from the filename (grouping similar names)
            label = os.path.splitext(video_file)[0]  # Remove the .mp4 extension
            
            # Grouping logic: Extract base name (e.g., 'car' from 'car2.mp4', 'car3.mp4')
            base_label = label.split('2')[0].split('3')[0]  # Modify based on your naming convention
            
            if keypoints.size > 0:  # Only add if keypoints are found
                if base_label not in data:
                    data[base_label] = []
                data[base_label].append(keypoints)
            else:
                print(f"No keypoints extracted from video: {video_path}")
    
    return data

# Example usage
data_directory = 'C:\\Users\\vladimir\\Desktop\\Projects\\NS\\data'  # Your directory
data = load_videos_from_directory(data_directory)

# Check the data structure
for label, sequences in data.items():
    print(f"Word: {label}, Number of Videos: {len(sequences)}")
